In [ ]:
"""Manski-Imbens Critical Value."""

from functools import partial

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.optimize import fsolve
from scipy.stats import norm

In [ ]:
# Parameters
sigma = 1
N = 1000

sigma_to_plot = [0.5, 1, 2]
n_to_plot = [100, 1000, 10000]

# All combinations
params_to_plot = [(s, n) for s in sigma_to_plot for n in n_to_plot]

In [ ]:
# Critical value two-sided is CDF evaluated at alpha = 0.95
alpha = 0.95
z_alpha = norm.ppf(alpha)

z_alpha_half = norm.ppf((alpha + 1) / 2)

z_alpha, z_alpha_half

In [ ]:
def imbens_manski_z(alpha: float, num_obs: int, p: float, sigma: float) -> float:
    """Imbens-Manski critical value."""

    # Defined as the root of
    def fun(x, p=p, alpha=alpha):
        return (
            norm.cdf(x + np.sqrt(num_obs) * (1 - p) / (sigma * np.sqrt(p)))
            - norm.cdf(-x)
            - alpha
        )

    # Find root
    return fsolve(fun, 1.96)[0]

In [ ]:
p_grid = np.linspace(0.8, 1.0, 100)

im_z = partial(imbens_manski_z, alpha=alpha)

In [ ]:
results = {}

for params in params_to_plot:
    results[params] = [im_z(p=p, num_obs=params[1], sigma=params[0]) for p in p_grid]

In [ ]:
fig = go.Figure()

for params, im_z_by_p in results.items():
    fig.add_trace(
        go.Scatter(
            x=p_grid,
            y=im_z_by_p,
            mode="lines",
            name=f"sigma={params[0]}, n={params[1]}",
        ),
    )

# Add horitzontal lines for z_alpha and z_alpha_half

fig.add_trace(
    go.Scatter(x=p_grid, y=[z_alpha] * len(p_grid), mode="lines", name="z_alpha"),
)

fig.add_trace(
    go.Scatter(
        x=p_grid,
        y=[z_alpha_half] * len(p_grid),
        mode="lines",
        name="z_alpha_half",
    ),
)


fig.update_layout(
    title="Imbens-Manski Critical Value",
    xaxis_title="p",
    yaxis_title="Critical Value",
)

fig.show()

In [ ]:
# Plot implied alpha of normal distribution
fig = go.Figure()

for params, im_z_by_p in results.items():
    fig.add_trace(
        go.Scatter(
            x=p_grid,
            y=1 - norm.cdf(im_z_by_p),
            mode="lines",
            name=f"sigma={params[0]}, n={params[1]}",
        ),
    )

fig.show()

In [ ]:
def _solve_imbens_manski_critival_value(
    alpha: float,
    n_obs: int,
    length: float,
    sigma_lower: float,
    sigma_upper: float,
) -> float:
    """Imbens-Manski critical value."""

    # Function defining the critival value (Equation 7 in IM 2004 ECMA).
    # Note we define CIs by 1 - alpha coverage, IM use alpha coverage, hence
    # we subtract (1 - alpha).
    def fun(x):
        return (
            norm.cdf(x + np.sqrt(n_obs) * length / max(sigma_lower, sigma_upper))
            - norm.cdf(-x)
            - (1 - alpha)
        )

    # Find root
    return fsolve(fun, 1.96)[0]

In [ ]:
crit = _solve_imbens_manski_critival_value(
    alpha=0.05,
    n_obs=10_000,
    length=0.01,
    sigma_lower=10,
    sigma_upper=10,
)

crit, 1 - norm.cdf(crit)

In [ ]:
fun = partial(
    _solve_imbens_manski_critival_value,
    alpha=0.05,
    n_obs=10_000,
    sigma_lower=10,
    sigma_upper=10,
)

length_grid = np.linspace(0.001, 0.1, 100)

results = [fun(length=length) for length in length_grid]

implied_alpha = [1 - norm.cdf(crit) for crit in results]

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=length_grid,
        y=results,
        mode="lines",
        name="Imbens-Manski Critical Value",
    ),
)

In [ ]:
data = pd.DataFrame({"length": length_grid, "crit": results, "alpha": implied_alpha})